## Import packages

In [1]:
import os
import sys
import re
import pandas as pd
import collections
from collections import Counter
import compress_pickle
import copy
import fnmatch
import itertools
import libsbml
import numpy as np
import time
import matplotlib.pyplot as plt
import string

# Import my local AMAS package
sys.path.insert(0, '/Users/luna/Desktop/CRBM/AMAS_proj/AMAS-v2')
from AMAS import annotation_maker as am
from AMAS import constants as cn
from AMAS import iterator as it
from AMAS import tools
from AMAS import species_annotation as sa
from AMAS import reaction_annotation as ra 
from AMAS import gene_annotation as ga
from AMAS import recommender

# Directory
biomd_model_dir = '/Users/luna/Desktop/CRBM/AMAS_proj/Models/BioModels/'
bigg_model_dir = '/Users/luna/Desktop/CRBM/AMAS_proj/Models/BiggModels/'
biodivine_model_dir = '/Users/luna/Desktop/CRBM/AMAS_proj/Models/BioDivine/'
output_dir = '/Users/luna/Desktop/CRBM/AMAS_proj/Results/'

In [4]:
ecoli_mg1655_models = ['e_coli_core.xml', 'iML1515.xml', 'iAF1260.xml','iJR904.xml','iAF1260b.xml','iJO1366.xml']
human_models = ['Recon3D.xml', 'iAB_RBC_283.xml','iAT_PLT_636.xml','RECON1.xml']

In [4]:
one_fpath = "/Users/luna/Desktop/CRBM/AMAS_proj/Models/BiggModels/e_coli_core.xml"
reader = libsbml.SBMLReader()
document = reader.readSBML(one_fpath)
model = document.getModel().getPlugin("fbc")
model.getNumGeneProducts()

137

In [ ]:
gene_product = fbc_plugin.getGeneProduct(3)
print(gene_product)
print(gene_product.getIdAttribute())
print(gene_product.getLabel())
print(gene_product.getName())

In [23]:
model_file = "iAM_Pf480.xml"
one_fpath = bigg_model_dir + model_file
recom = recommender.Recommender(libsbml_fpath=one_fpath)
geneids2evaluate = list(recom.genes.exist_annotation.keys())
gene_id = []
gene_name = []
predictions = []
predictions_labels = []
num_cands = []
match_scores = []
recall = []
precision = []
tax_dict_file = bigg_model_dir + "bigg_organisms.csv"
tax_id, tax_name = get_model_taxonomy(model_file, tax_dict_file)
recoms = recom.getGeneListRecommendation(pred_ids=geneids2evaluate,tax_id=tax_id)
recalls = recom.getGeneStatistics(tax_id=tax_id,model_mean=False)['recall']
precisions = recom.getGeneStatistics(tax_id=tax_id,model_mean=False)['precision']
for one_recom in recoms:
    one_spec = one_recom.id
    gene_id.append(one_spec)
    gene_name.append(recom.genes.getNameToUse(one_spec))
    predictions.append(one_recom.candidates)
    predictions_labels.append(one_recom.labels)
    num_cands.append(len(one_recom.candidates))
    match_scores.append(one_recom.candidates[0][1])
    recall.append(recalls[one_spec])
    precision.append(precisions[one_spec])
spec_df = pd.DataFrame({
                    'gene_id': gene_id,
                    'gene_name': gene_name,
                    'predictions': predictions,
                    'predictions_labels': predictions_labels,
                    'num_candidates': num_cands,
                    'match_score': match_scores,
                    'recall': recall,
                    'precision': precision})
spec_df.to_csv(output_dir + model_file + ".csv", index=False)

,gene_id,gene_name,predictions,predictions_labels,num_candidates,match_score,recall,precision
0,G_PF3D7_0507200,PF3D7_0507200,"[(812667, 1.0), (812844, 1.0), (812882, 1.0), ...","[PF3D7_0207500, PF3D7_0502700, PF3D7_0507200, ...",9,1.0,1.0,0.111
1,G_PF3D7_0932300,PF3D7_0932300,"[(813509, 1.0), (813594, 1.0), (813602, 1.0), ...","[PF3D7_0923300, PF3D7_0932300, PF3D7_0933200, ...",4,1.0,1.0,0.250
2,G_PF3D7_1360800,PF3D7_1360800,"[(810517, 1.0), (810533, 1.0), (813907, 1.0), ...","[PF3D7_1036800, PF3D7_1038600, PF3D7_1306800, ...",15,1.0,1.0,0.067
3,G_PF3D7_0409500,PF3D7_0409500,"[(812389, 1.0), (812414, 1.0), (812845, 1.0), ...","[PF3D7_0409500, PF3D7_0405900, PF3D7_0509400, ...",6,1.0,1.0,0.167
4,G_PF3D7_1446200,PF3D7_1446200,"[(811480, 1.0), (811497, 1.0), (811986, 1.0), ...","[PF3D7_1244600, PF3D7_1246400, PF3D7_1442600, ...",8,1.0,1.0,0.125
...,...,...,...,...,...,...,...,...
455,G_PF3D7_1115300,PF3D7_1115300,"[(810690, 1.0), (810708, 1.0), (810872, 1.0), ...","[PF3D7_1113500, PF3D7_1115300, PF3D7_1131500, ...",7,1.0,1.0,0.143
456,G_PF3D7_1115700,PF3D7_1115700,"[(810712, 1.0), (810728, 1.0)]","[PF3D7_1115700, PF3D7_1117500]",2,1.0,1.0,0.500
457,G_PF3D7_1407800,PF3D7_1407800,"[(811657, 1.0), (811665, 1.0), (812257, 1.0), ...","[PF3D7_1407800, PF3D7_1408700, PF3D7_1470800, ...",10,1.0,1.0,0.100
458,G_PF3D7_1311800,PF3D7_1311800,"[(810693, 1.0), (810736, 1.0), (810874, 1.0), ...","[PF3D7_1113800, PF3D7_1118300, PF3D7_1131800, ...",6,1.0,1.0,0.167


## Test on Gene Predictions  
### Bigg Models

In [7]:
def generate_model_genes_statistics(model_dir, output_csv):
    model_statistics_data = []

    # All SBML files in the model directory
    model_files = [f for f in os.listdir(model_dir) if f.endswith('.xml')]

    for idx, model_file in enumerate(model_files):
        print(f"We are at {idx}: Processing {model_file}")
        one_fpath = os.path.join(model_dir, model_file)
        reader = libsbml.SBMLReader()
        document = reader.readSBML(one_fpath)
        model = document.getModel()
        if model is None:
            print(f"Skipping {model_file} (invalid model)")
            continue
        fbc_plugin = model.getPlugin("fbc")
        if fbc_plugin is None:
            print(f"Skipping {model_file} (no fbc package)")
            continue
        recom = recommender.Recommender(libsbml_fpath=one_fpath)

        # Get species statistics
        num_genes = fbc_plugin.getNumGeneProducts()
        genes2evaluate = list(recom.genes.exist_annotation.keys())
        num_annotated_genes = len(genes2evaluate)
        annotated = num_annotated_genes > 0
        
        # Append the results for this species
        model_statistics_data.append({
            'model': model_file,
            'annotated': annotated,
            'num_genes': num_genes,
            'num_annotated_genes': num_annotated_genes
        })


    # Create a DataFrame from the collected data
    df = pd.DataFrame(model_statistics_data)

    # Save the DataFrame to a CSV file
    df.to_csv(output_csv, index=False)

output_csv = output_dir + "bigg_model_gene_statistics.csv"
generate_model_genes_statistics(bigg_model_dir, output_csv)

We are at 0: Processing iCHOv1.xml
We are at 1: Processing iYL1228.xml
We are at 2: Processing iIS312.xml
We are at 3: Processing iSDY_1059.xml
We are at 4: Processing iEC1356_Bl21DE3.xml
We are at 5: Processing iECP_1309.xml
We are at 6: Processing iJN746.xml
We are at 7: Processing iJR904.xml
We are at 8: Processing iJB785.xml
We are at 9: Processing iY75_1357.xml
We are at 10: Processing iEcHS_1320.xml
We are at 11: Processing iUTI89_1310.xml
We are at 12: Processing iB21_1397.xml
We are at 13: Processing iSbBS512_1146.xml
We are at 14: Processing iNF517.xml
We are at 15: Processing iNJ661.xml
We are at 16: Processing iECIAI39_1322.xml
Skipping iECIAI39_1322.xml (invalid model)
We are at 17: Processing iECO103_1326.xml
We are at 18: Processing iSSON_1240.xml
We are at 19: Processing iCHOv1_DG44.xml
We are at 20: Processing iAB_RBC_283.xml
We are at 21: Processing iAM_Pk459.xml
We are at 22: Processing iPC815.xml
We are at 23: Processing iAPECO1_1312.xml
We are at 24: Processing iMM1

In [40]:
bigg_model_gene_statistics = pd.read_csv(output_dir + "bigg_model_gene_statistics.csv")
annotated_models = bigg_model_gene_statistics[bigg_model_gene_statistics['num_annotated_genes'] != 0]
print(f"Average number of genes: {annotated_models['num_genes'].mean():.1f}")
print(f"Average number of annotated genes: {annotated_models['num_annotated_genes'].mean():.1f}")
print(f"There are {len(annotated_models)} models with annotated genes:")
annotated_models


Average number of genes: 987.6
Average number of annotated genes: 904.9
There are 28 models with annotated genes:


,model,annotated,num_genes,num_annotated_genes
0,iCHOv1.xml,True,1766,1743
6,iJN746.xml,True,746,729
7,iJR904.xml,True,904,892
15,iNJ661.xml,True,661,655
18,iCHOv1_DG44.xml,True,1184,1168
19,iAB_RBC_283.xml,True,346,311
21,iPC815.xml,True,815,814
23,iMM1415.xml,True,1375,1344
25,iJO1366.xml,True,1367,1366
35,iAF1260.xml,True,1261,1260


In [85]:
def get_model_taxonomy(model_file, tax_dict_file):
    tax_dict_df = pd.read_csv(tax_dict_file)
    model_file_id = model_file.replace('.xml', '')
    tax_id = tax_dict_df[tax_dict_df['id'] == model_file_id]['tax_id'].values[0]
    tax_name = tax_dict_df[tax_dict_df['id'] == model_file_id]['organism'].values[0]
    return tax_id, tax_name

def generate_gene_accuracy(model_dir, output_csv, tax_dict_file):
    model_files = [f for f in os.listdir(model_dir) if f.endswith('.xml')]
    model_names = []
    taxonomy_id = []
    taxonomy_name = []
    gene_id = []
    gene_name = []
    len_names = []
    predictions = []
    predictions_labels = []
    num_cands = []
    match_scores = []
    recall = []
    precision = []
    is_correct_included = []
    processing_time = []

    for idx,model_file in enumerate(model_files):
        print(f"We are at {idx}/{len(model_files)}: Processing {model_file}")
        one_fpath = os.path.join(model_dir, model_file)
        tax_id, tax_name = get_model_taxonomy(model_file, tax_dict_file)
        try:
            recom = recommender.Recommender(libsbml_fpath=one_fpath)
            if recom is None or recom.genes is None:
                print(f"Skipping {model_file} (invalid recommender or genes)")
                continue
            genes2evaluate = list(recom.genes.exist_annotation.keys())
            # run algorithm if there is at least one species to evaluate
            if genes2evaluate:
                time_start = time.time()
                recoms = recom.getGeneListRecommendation(tax_id=tax_id,pred_ids=genes2evaluate)
                time_end = time.time()
                recalls = recom.getGeneStatistics(tax_id=tax_id,model_mean=False)['recall']
                precisions = recom.getGeneStatistics(tax_id=tax_id,model_mean=False)['precision']
                for one_recom in recoms:
                    model_names.append(model_file)
                    taxonomy_id.append(tax_id)
                    taxonomy_name.append(tax_name)
                    one_spec = one_recom.id
                    gene_id.append(one_spec)
                    gene_name.append(recom.genes.getNameToUse(one_spec))
                    len_names.append(len(gene_name[-1]))
                    predictions.append(one_recom.candidates)
                    predictions_labels.append(one_recom.labels)
                    num_cands.append(len(one_recom.candidates))
                    match_scores.append(one_recom.candidates[0][1])
                    recall.append(recalls[one_spec])
                    precision.append(precisions[one_spec])
                    processing_time.append(time_end - time_start)
                    if recall[-1] > 0:
                        is_correct_included.append(1)
                    else:
                        is_correct_included.append(0)
        except Exception as e:
            print(f"Error processing {model_file}: {str(e)}")
            continue

    # Create a DataFrame from the collected data
    spec_df = pd.DataFrame({'model': model_names,
                        'taxonomy_id': taxonomy_id,
                        'taxonomy_name': taxonomy_name,
                        'gene_id': gene_id,
                        'gene_name': gene_name,
                        'name_length': len_names,
                        'predictions': predictions,
                        'predictions_labels': predictions_labels,
                        'num_candidates': num_cands,
                        'match_score': match_scores,
                        'recall': recall,
                        'precision': precision,
                        'accuracy': is_correct_included,
                        'processing_time': processing_time})

    # Save the DataFrame to a CSV file
    spec_df.to_csv(output_csv, index=False)

output_csv = output_dir + "bigg_model_gene_accuracy_protein_coding.csv"
tax_dict_file = bigg_model_dir + "bigg_organisms.csv"
generate_gene_accuracy(bigg_model_dir, output_csv, tax_dict_file)

We are at 0/108: Processing iCHOv1.xml
We are at 1/108: Processing iYL1228.xml
We are at 2/108: Processing iIS312.xml
We are at 3/108: Processing iSDY_1059.xml
We are at 4/108: Processing iEC1356_Bl21DE3.xml
We are at 5/108: Processing iECP_1309.xml
We are at 6/108: Processing iJN746.xml
Error processing iJN746.xml: Reference data not found for tax_id 160488
We are at 7/108: Processing iJR904.xml
We are at 8/108: Processing iJB785.xml
We are at 9/108: Processing iY75_1357.xml
We are at 10/108: Processing iEcHS_1320.xml
We are at 11/108: Processing iUTI89_1310.xml
We are at 12/108: Processing iB21_1397.xml
We are at 13/108: Processing iSbBS512_1146.xml
We are at 14/108: Processing iNF517.xml
We are at 15/108: Processing iNJ661.xml
We are at 16/108: Processing iECIAI39_1322.xml
Error processing iECIAI39_1322.xml: 'NoneType' object has no attribute 'keys'
We are at 17/108: Processing iECO103_1326.xml
We are at 18/108: Processing iSSON_1240.xml
We are at 19/108: Processing iCHOv1_DG44.xml


In [87]:
def safe_eval_with_eval(x):
    try:
        return eval(x) if isinstance(x, str) else []
    except Exception as e:
        print(f"Failed to parse: {x} with error: {e}")
        return []
def print_model_accuracy(df, show_organism=False):
    if show_organism:
        print("Number of organisms assessed: %d" % df['taxonomy_name'].nunique())
        for one_organism in df['taxonomy_name'].unique():
            print(f"  - {one_organism}")
            df_organism = df[df['taxonomy_name'] == one_organism]
            print("Number of models assessed: %d" % df_organism['model'].nunique())
            print("Number of genes assessed: %d" % df_organism['gene_id'].nunique())
            print("Average recall (per model) : %.003f" % df_organism.groupby('model')['recall'].mean().mean())
            print("Average precision (per model) : %.003f" % df_organism.groupby('model')['precision'].mean().mean())
            print("Average match score (per model): %.003f" % df_organism.groupby('model')['match_score'].mean().mean())
            print("Average accuracy (per model): %.003f" % df_organism.groupby('model')['accuracy'].mean().mean())
            df_copy = df_organism.copy()
            df_copy['parsed_predictions'] = df_copy['predictions'].apply(safe_eval_with_eval)
            df_copy['num_predictions'] = df_copy['parsed_predictions'].apply(len)
            average_predictions = df_copy['num_predictions'].mean()
            print(f"Average number of predictions per row: %.003f" % average_predictions)
            print("__________________________________________________________")
    else:
        print("Number of models assessed: %d" % df['model'].nunique())
        print("Number of genes assessed: %d" % df['gene_id'].nunique())
        print("Average recall (per model) : %.003f" % df.groupby('model')['recall'].mean().mean())
        print("Average precision (per model) : %.003f" % df.groupby('model')['precision'].mean().mean())
        print("Average match score (per model): %.003f" % df.groupby('model')['match_score'].mean().mean())
        print("Average accuracy (per model): %.003f" % df.groupby('model')['accuracy'].mean().mean())
    # Fixed processing time calculations
    mean_processing_time = df.groupby('model')['processing_time'].first().mean()
    print("Ave. processing time (per model): %.003f" % mean_processing_time)
    # Calculate per element processing time
    num_elements = df.groupby('model').size().mean()
    mean_processing_time_per_element = mean_processing_time / num_elements
    print("Ave. processing time (per element, per model): %.003f" % mean_processing_time_per_element)
    # Average number of predictions per species
    df = df.copy()
    df['parsed_predictions'] = df['predictions'].apply(safe_eval_with_eval)
    df['num_predictions'] = df['parsed_predictions'].apply(len)
    average_predictions = df['num_predictions'].mean()
    print(f"Average number of predictions per row: %.003f" % average_predictions)
    
bigg_model_gene_accuracy = pd.read_csv(output_dir + "bigg_model_gene_accuracy_protein_coding.csv")
print("\n\n<Bigg Genes (All)>")
print_model_accuracy(bigg_model_gene_accuracy)



<Bigg Genes (All)>
Number of models assessed: 22
Number of genes assessed: 12322
Average recall (per model) : 0.906
Average precision (per model) : 0.604
Average match score (per model): 0.928
Average accuracy (per model): 0.906
Ave. processing time (per model): 2.794
Ave. processing time (per element, per model): 0.003
Average number of predictions per row: 2.345


In [88]:
discontinued = ['Plasmodium falciparum 3D7','Plasmodium cynomolgi strain B','Pseudomonas putida KT2440','Yersinia pestis CO92','Helicobacter pylori 26695','Thermotoga maritima MSB8','Plasmodium vivax Sal-1','Plasmodium berghei']
bigg_model_gene_accuracy = pd.read_csv(output_dir+"bigg_model_gene_accuracy_protein_coding.csv")
bigg_model_gene_accuracy = bigg_model_gene_accuracy[~bigg_model_gene_accuracy['taxonomy_name'].isin(discontinued)]
print_model_accuracy(bigg_model_gene_accuracy)

Number of models assessed: 18
Number of genes assessed: 10519
Average recall (per model) : 0.996
Average precision (per model) : 0.720
Average match score (per model): 1.000
Average accuracy (per model): 0.996
Ave. processing time (per model): 3.327
Ave. processing time (per element, per model): 0.003
Average number of predictions per row: 2.050


In [89]:
print_model_accuracy(bigg_model_gene_accuracy, show_organism=True)

Number of organisms assessed: 7
  - Cricetulus griseus
Number of models assessed: 2
Number of genes assessed: 1743
Average recall (per model) : 0.984
Average precision (per model) : 0.776
Average match score (per model): 1.000
Average accuracy (per model): 0.984
Average number of predictions per row: 1.962
__________________________________________________________
  - Escherichia coli str. K-12 substr. MG1655
Number of models assessed: 6
Number of genes assessed: 1553
Average recall (per model) : 0.999
Average precision (per model) : 0.727
Average match score (per model): 1.000
Average accuracy (per model): 0.999
Average number of predictions per row: 1.867
__________________________________________________________
  - Mycobacterium tuberculosis H37Rv
Number of models assessed: 2
Number of genes assessed: 1014
Average recall (per model) : 1.000
Average precision (per model) : 0.856
Average match score (per model): 1.000
Average accuracy (per model): 1.000
Average number of predictions 

In [90]:
human = bigg_model_gene_accuracy[bigg_model_gene_accuracy['taxonomy_name'] == 'Homo sapiens']
human[human['recall'] != 1]

,model,taxonomy_id,taxonomy_name,gene_id,gene_name,name_length,predictions,predictions_labels,num_candidates,match_score,recall,precision,accuracy,processing_time
12430,Recon3D.xml,9606,Homo sapiens,G_55350_AT3,VNN3,4,"[('7448', np.float64(0.866))]",['VTN'],1,0.866,0.0,0.0,0,11.421148
12441,Recon3D.xml,9606,Homo sapiens,G_55350_AT1,VNN3,4,"[('7448', np.float64(0.866))]",['VTN'],1,0.866,0.0,0.0,0,11.421148
12446,Recon3D.xml,9606,Homo sapiens,G_55350_AT2,VNN3,4,"[('7448', np.float64(0.866))]",['VTN'],1,0.866,0.0,0.0,0,11.421148
12695,Recon3D.xml,9606,Homo sapiens,G_728441_AT1,GGT2,4,"[('102724197', np.float64(1.0))]",['LOC102724197'],1,1.000,0.0,0.0,0,11.421148
13243,Recon3D.xml,9606,Homo sapiens,G_200810_AT1,ALG1L,5,"[('2657', np.float64(0.945)), ('3956', np.floa...","['GDF1', 'LGALS1', 'S100A8', 'S100A9', 'CCL4',...",8,0.945,0.0,0.0,0,11.421148
17029,RECON1.xml,9606,Homo sapiens,G_55350_AT3,VNN3,4,"[('7448', np.float64(0.866))]",['VTN'],1,0.866,0.0,0.0,0,9.244529
17030,RECON1.xml,9606,Homo sapiens,G_55350_AT2,VNN3,4,"[('7448', np.float64(0.866))]",['VTN'],1,0.866,0.0,0.0,0,9.244529
17038,RECON1.xml,9606,Homo sapiens,G_55350_AT1,VNN3,4,"[('7448', np.float64(0.866))]",['VTN'],1,0.866,0.0,0.0,0,9.244529


In [42]:
# Find rows where recall is 0
zero_recall_df = bigg_model_gene_accuracy[bigg_model_gene_accuracy['recall'] == 0]
print("\n\nNumber of genes with zero recall: %d" % len(zero_recall_df))
zero_recall_df



Number of genes with zero recall: 3


,model,taxonomy,gene_id,gene_name,name_length,predictions,num_candidates,match_score,recall,precision,accuracy,processing_time
5117,iML1515.xml,ecoli_mg1655,G_b0624,flc,3,"[(946563, np.float64(1.0))]",1,1.0,0.0,0.0,0,1.941852
5159,iML1515.xml,ecoli_mg1655,G_b2245,rhmA,4,"[(948402, np.float64(1.0))]",1,1.0,0.0,0.0,0,1.941852
5177,iML1515.xml,ecoli_mg1655,G_b4104,phnE,4,"[(38094977, np.float64(1.0))]",1,1.0,0.0,0.0,0,1.941852


### Biomodels

In [37]:
dir = "/Users/luna/Desktop/CRBM/AMAS_proj/Models/BioModels/ecoli_mg1655"
ecoli_mg1655_models = [f for f in os.listdir(dir) if f.endswith('.xml')]
output_csv = output_dir + "biomd_model_gene_accuracy_ecoli.csv"
generate_gene_accuracy(dir, output_csv)

['BIOMD0000001062.xml']
We are at 0/1: Processing BIOMD0000001062.xml


In [54]:
biomd_model_gene_accuracy_ecoli = pd.read_csv(output_dir + "biomd_model_gene_accuracy_ecoli.csv")
print_model_accuracy(biomd_model_gene_accuracy_ecoli)

Number of models assessed: 1
Number of genes assessed: 1371
Average recall (per model) : 0.998
Average precision (per model) : 0.706
Average match score (per model): 1.000
Average accuracy (per model): 0.998
Ave. processing time (per model): 1.596
Ave. processing time (per element, per model): 0.001
Average number of predictions per row: 1.9438366156090445


In [55]:
# Find rows where recall is 0
zero_recall_df = biomd_model_gene_accuracy_ecoli[biomd_model_gene_accuracy_ecoli['recall'] == 0]
print("\n\nNumber of genes with zero recall: %d" % len(zero_recall_df))
zero_recall_df



Number of genes with zero recall: 3


,model,taxonomy,gene_id,gene_name,name_length,predictions,predictions_labels,num_candidates,match_score,recall,precision,accuracy,processing_time
1278,BIOMD0000001062.xml,ecoli_mg1655,G_ECOLIN_RS03405,flc,3,"[(946563, np.float64(1.0))]",['fcl'],1,1.0,0.0,0.0,0,1.596149
1308,BIOMD0000001062.xml,ecoli_mg1655,G_ECOLIN_RS13085,rhmA,4,"[(948402, np.float64(1.0))]",['rhaM'],1,1.0,0.0,0.0,0,1.596149
1355,BIOMD0000001062.xml,ecoli_mg1655,G_ECOLIN_RS23755,phnE,4,"[(38094977, np.float64(1.0))]",['phnE'],1,1.0,0.0,0.0,0,1.596149


## Statistics of Gene annotations
### Bigg Models 

In [24]:
def getOntologyFromString(string_annotation,
                          bqbiol_qualifiers=['is', 'isVersionOf']):
  """
  Parse string and return string annotation,
  marked as <bqbiol:is> or <bqbiol:isVersionOf>.
  If neither exists, return None.

  Parameters
  ----------
  string_annotation: str
  bqbiol_qualifiers: str-list
      Use 'is' and 'isVersionOf' by default
  

  Returns
  -------
  list-tuple (ontology type, ontology id)
       Return [] if none is provided
  """
  combined_str = ''
  for one_qualifier in bqbiol_qualifiers:
    one_match = '<bqbiol:' + one_qualifier + \
                '[^a-zA-Z].*?<\/bqbiol:' + \
                one_qualifier + '>'
    one_matched = re.findall(one_match,
                  string_annotation,
                  flags=re.DOTALL)
    if len(one_matched)>0:
      matched_filt = [s.replace("      ", "") for s in one_matched]
      one_str = '\n'.join(matched_filt) 
    else:
      one_str = ''
    combined_str = combined_str + one_str
  identifiers_list = re.findall('identifiers\.org/.*/', combined_str)
  result_identifiers = [(r.split('/')[1],r.split('/')[2].replace('\"', '')) \
                        for r in identifiers_list]
  return result_identifiers

def collect_metabolites_statistics(model_dir, output_csv, model_ontology_csv):
    """
    Collect statistics of ontology types from SBML models in a folder.

    Parameters:
    - folder_path (str): Path to the folder containing SBML models.
    - output_csv (str): Path to save overall ontology statistics as a CSV.
    - model_ontology_csv (str): Path to save per-model ontology statistics as a CSV.
    """
    overall_counter = Counter()
    model_data = []

    # Iterate over all SBML files in the folder
    model_files = [f for f in os.listdir(model_dir) if f.endswith('.xml')]

    for idx, file_name in enumerate(model_files):
      print(f"We are at {idx}: Processing {file_name}")

      # Read the SBML model
      file_path = os.path.join(model_dir, file_name)
      reader = libsbml.SBMLReader()
      document = reader.readSBML(file_path)
      model = document.getModel()

      if model is None:
          print(f"Skipping {file_name} (invalid model)")
          continue

      # Count ontology types for the current model
      model_counter = Counter()

      # Iterate over all species in the model
      for species in model.getListOfSpecies():
          annotation_string = species.getAnnotationString()
          resources = getOntologyFromString(annotation_string)

          # Count ontology types
          for ontology_type, ontology_id in resources:
              overall_counter[ontology_type] += 1
              model_counter[ontology_type] += 1

      # Store model-level data
      for ontology_type, count in model_counter.items():
          model_data.append({
              'model': file_name,
              'ontology_type': ontology_type,
              'count': count
          })

    # Convert overall statistics to a DataFrame
    overall_stats_df = pd.DataFrame.from_dict(overall_counter, orient='index', columns=['count'])
    overall_stats_df.reset_index(inplace=True)
    overall_stats_df.columns = ['ontology_type', 'count']

    # Convert model-specific data to a DataFrame
    model_stats_df = pd.DataFrame(model_data)

    # Save the dataframes to CSV
    overall_stats_df.to_csv(output_csv, index=False)
    model_stats_df.to_csv(model_ontology_csv, index=False)
    print(f"Overall ontology statistics saved to {output_csv}")
    print(f"Model-specific ontology statistics saved to {model_ontology_csv}")


def get_gene_annotation_string(sbml_file):
    """
    Extract GeneProducts annotatino string from an SBML model.

    Parameters:
    - sbml_file (str): Path to the SBML file.

    Returns:
    - list: List of GeneProduct annotation strings.
    """
    # Read the SBML model
    reader = libsbml.SBMLReader()
    document = reader.readSBML(sbml_file)
    model = document.getModel()
    if model is None:
        print(f"Failed to read the model: {sbml_file}")
        return []
        
    # Check if the model uses the FBC package
    fbc_plugin = model.getPlugin("fbc")
    if not fbc_plugin:
        print(f"The model does not use the FBC package: {sbml_file}")
        return []

    # Get GeneProducts from the FBC plugin
    gene_annotations = []
    for i in range(fbc_plugin.getNumGeneProducts()):
        gene_product = fbc_plugin.getGeneProduct(i)
        annotation_string = gene_product.getAnnotationString()
        gene_annotations.append(annotation_string)
    return gene_annotations

def collect_gene_product_statistics(model_dir, overall_output_csv, model_ontology_csv):
    """
    Collect statistics of GeneProducts from SBML models in a folder.

    Parameters:
    - folder_path (str): Path to the folder containing SBML models.
    - overall_output_csv (str): Path to save overall GeneProduct statistics as a CSV.
    - model_ontology_csv (str): Path to save per-model GeneProduct statistics as a CSV.
    """
    overall_counter = Counter()
    model_data = []

    # Iterate over all SBML files in the folder
    model_files = [f for f in os.listdir(model_dir) if f.endswith('.xml')]
    for idx, file_name in enumerate(model_files):
      print(f"We are at {idx}: Processing {file_name}")
      file_path = os.path.join(model_dir, file_name)
      gene_annotations = get_gene_annotation_string(file_path)

      model_counter = Counter()
      for gene_annotation in gene_annotations:
        resources = getOntologyFromString(gene_annotation)

        # Count ontology types
        for ontology_type, ontology_id in resources:
            overall_counter[ontology_type] += 1
            model_counter[ontology_type] += 1

      # Store model-level data
      for ontology_type, count in model_counter.items():
        model_data.append({
            'model': file_name,
            'ontology_type': ontology_type,
            'count': count
        })

    # Convert overall statistics to a DataFrame
    overall_stats_df = pd.DataFrame.from_dict(overall_counter, orient='index', columns=['count'])
    overall_stats_df.reset_index(inplace=True)
    overall_stats_df.columns = ['ontology_type', 'count']

    # Convert model-specific data to a DataFrame
    model_stats_df = pd.DataFrame(model_data)

    # Save the dataframes to CSV
    overall_stats_df.to_csv(output_csv, index=False)
    model_stats_df.to_csv(model_ontology_csv, index=False)
    print(f"Overall GeneProduct statistics saved to {overall_output_csv}")
    print(f"Model-specific GeneProduct statistics saved to {model_ontology_csv}")

In [22]:
output_csv = "/Users/luna/Desktop/CRBM/AMAS_proj/Results/bigg_ontology_statistics.csv"
model_ontology_csv = "/Users/luna/Desktop/CRBM/AMAS_proj/Results/bigg_model_ontology_statistics.csv"
collect_metabolites_statistics(bigg_model_dir, output_csv, model_ontology_csv)

We are at 0
We are at 1
We are at 2
We are at 3
We are at 4
We are at 5
We are at 6
We are at 7
We are at 8
We are at 9
We are at 10
We are at 11
We are at 12
We are at 13
We are at 14
We are at 15
We are at 16
We are at 17
We are at 18
We are at 19
We are at 20
We are at 21
We are at 22
We are at 23
We are at 24
We are at 25
We are at 26
We are at 27
We are at 28
We are at 29
We are at 30
We are at 31
We are at 32
We are at 33
We are at 34
We are at 35
We are at 36
We are at 37
We are at 38
We are at 39
We are at 40
We are at 41
We are at 42
We are at 43
We are at 44
We are at 45
We are at 46
We are at 47
We are at 48
We are at 49
We are at 50
We are at 51
We are at 52
We are at 53
We are at 54
We are at 55
We are at 56
We are at 57
We are at 58
We are at 59
We are at 60
We are at 61
We are at 62
We are at 63
We are at 64
We are at 65
We are at 66
We are at 67
We are at 68
We are at 69
We are at 70
We are at 71
We are at 72
We are at 73
We are at 74
We are at 75
We are at 76
We are at

In [60]:
bigg_metabolites_ontology_statistics = pd.read_csv("/Users/luna/Desktop/CRBM/AMAS_proj/Results/bigg_metabolites_ontology_statistics.csv")
bigg_metabolites_ontology_statistics.sort_values(by='count', ascending=False)

,ontology_type,count
3,chebi,881182
0,bigg.metabolite,180802
10,seed.compound,177255
11,reactome.compound,176773
1,metanetx.chemical,173772
2,biocyc,151278
5,hmdb,142708
7,kegg.compound,140296
6,inchi_key,134440
9,sabiork,124143


In [56]:
output_csv = "/Users/luna/Desktop/CRBM/AMAS_proj/Results/bigg_genes_ontology_statistics.csv"
model_ontology_csv = "/Users/luna/Desktop/CRBM/AMAS_proj/Results/bigg_genes_model_ontology_statistics.csv"
collect_gene_product_statistics(bigg_model_dir, output_csv, model_ontology_csv)

We are at 0: Processing iCHOv1.xml
We are at 1: Processing iYL1228.xml
We are at 2: Processing iIS312.xml
We are at 3: Processing iSDY_1059.xml
We are at 4: Processing iEC1356_Bl21DE3.xml
We are at 5: Processing iECP_1309.xml
We are at 6: Processing iJN746.xml
We are at 7: Processing iJR904.xml
We are at 8: Processing iJB785.xml
We are at 9: Processing iY75_1357.xml
We are at 10: Processing iEcHS_1320.xml
We are at 11: Processing iUTI89_1310.xml
We are at 12: Processing iB21_1397.xml
We are at 13: Processing iSbBS512_1146.xml
We are at 14: Processing iNF517.xml
We are at 15: Processing iNJ661.xml
We are at 16: Processing iECIAI39_1322.xml
Failed to read the model: /Users/luna/Desktop/CRBM/AMAS_proj/Models/BiggModels/iECIAI39_1322.xml
We are at 17: Processing iECO103_1326.xml
We are at 18: Processing iSSON_1240.xml
We are at 19: Processing iCHOv1_DG44.xml
We are at 20: Processing iAB_RBC_283.xml
We are at 21: Processing iAM_Pk459.xml
We are at 22: Processing iPC815.xml
We are at 23: Pro

In [58]:
bigg_genes_ontology_statistics = pd.read_csv("/Users/luna/Desktop/CRBM/AMAS_proj/Results/bigg_genes_ontology_statistics.csv")
bigg_genes_ontology_statistics.sort_values(by='count', ascending=False)

,ontology_type,count
1,ncbigi,102867
4,refseq_locus_tag,101721
2,refseq_name,77574
13,interpro,70094
3,refseq_synonym,45041
0,ncbigene,25336
5,refseq_old_locus_tag,23578
9,uniprot,18857
12,goa,14134
10,EnsemblGenomes-Gn,12622


### BioModels

In [25]:
output_csv = "/Users/luna/Desktop/CRBM/AMAS_proj/Results/biomd_genes_ontology_statistics.csv"
model_ontology_csv = "/Users/luna/Desktop/CRBM/AMAS_proj/Results/biomd_genes_model_ontology_statistics.csv"
collect_gene_product_statistics(biomd_model_dir, output_csv, model_ontology_csv)

We are at 0: Processing BIOMD0000000634.xml
The model does not use the FBC package: /Users/luna/Desktop/CRBM/AMAS_proj/Models/BioModels/BIOMD0000000634.xml
We are at 1: Processing BIOMD0000000152.xml
The model does not use the FBC package: /Users/luna/Desktop/CRBM/AMAS_proj/Models/BioModels/BIOMD0000000152.xml
We are at 2: Processing BIOMD0000000146.xml
The model does not use the FBC package: /Users/luna/Desktop/CRBM/AMAS_proj/Models/BioModels/BIOMD0000000146.xml
We are at 3: Processing BIOMD0000000620.xml
The model does not use the FBC package: /Users/luna/Desktop/CRBM/AMAS_proj/Models/BioModels/BIOMD0000000620.xml
We are at 4: Processing BIOMD0000000608.xml
The model does not use the FBC package: /Users/luna/Desktop/CRBM/AMAS_proj/Models/BioModels/BIOMD0000000608.xml
We are at 5: Processing BIOMD0000000191.xml
The model does not use the FBC package: /Users/luna/Desktop/CRBM/AMAS_proj/Models/BioModels/BIOMD0000000191.xml
We are at 6: Processing BIOMD0000000185.xml
The model does not u

In [52]:
bigg_genes_ontology_statistics = pd.read_csv("/Users/luna/Desktop/CRBM/AMAS_proj/Results/biomd_genes_model_ontology_statistics.csv")
bigg_genes_ontology_statistics.sort_values(by='count', ascending=False)

,model,ontology_type,count
6,BIOMD0000001062.xml,refseq_synonym,4081
4,BIOMD0000001062.xml,refseq_locus_tag,1433
3,BIOMD0000001062.xml,ncbigi,1431
5,BIOMD0000001062.xml,refseq_name,1373
0,BIOMD0000001062.xml,asap,1372
1,BIOMD0000001062.xml,ecogene,1371
2,BIOMD0000001062.xml,ncbigene,1371
7,BIOMD0000001062.xml,uniprot,1371
8,BIOMD0000001062.xml,refseq_old_locus_tag,62
9,BIOMD0000001046.xml,kegg.genes,25


## Create reference files for NCBI Gene
### For all organisms

Data are obtained from the NCBI gene FTP site: https://ftp.ncbi.nih.gov/gene/DATA/GENE_INFO/.

In [97]:
# All gene data
input_file = "/Users/luna/Desktop/CRBM/AMAS_proj/AMAS-v2/AMAS/files/ncbi/All_Data.gene_info.gz"
columns_to_keep = ["#tax_id", "GeneID", "Symbol", "Synonyms","type_of_gene"]
df = pd.read_csv(input_file, compression='gzip', sep='\t',usecols=columns_to_keep)
df.head()

,#tax_id,GeneID,Symbol,Synonyms,type_of_gene
0,7,5692769,NEWENTRY,-,other
1,9,2827857,NEWENTRY,-,other
2,11,10823747,NEWENTRY,-,other
3,14,6951813,NEWENTRY,-,other
4,19,3758873,NEWENTRY,-,other


In [98]:
# Exclude certain types of genes
print(df['type_of_gene'].value_counts())
# exclude_types = ['biological-region', 'unknown', 'other']
# df = df[~df['type_of_gene'].isin(exclude_types) & (df['Symbol'] != 'NEWENTRY')]

# Only include protein-coding genes
include_types = ['protein-coding']
df = df[df['type_of_gene'].isin(include_types)]


type_of_gene
protein-coding       46235835
ncRNA                 3480011
tRNA                  3033345
pseudo                2737966
rRNA                   995495
snRNA                  586439
snoRNA                 505390
biological-region      168474
other                  108372
unknown                 61023
miscRNA                  5128
scRNA                      21
Name: count, dtype: int64


In [99]:
output_dir = "/Users/luna/Desktop/CRBM/AMAS_proj/AMAS-v2/AMAS/files/ncbi/"
output_gene2label = "ncbigene2label_all_protein_coding.lzma"
gene_id_to_symbol = dict(zip(df['GeneID'], df['Symbol']))
with open(os.path.join(output_dir,output_gene2label), 'wb') as handle:
    compress_pickle.dump(gene_id_to_symbol, handle, compression="lzma", set_default_extension=False)
print(f"GeneID to Symbol saved to {output_dir+output_gene2label}")

GeneID to Symbol saved to /Users/luna/Desktop/CRBM/AMAS_proj/AMAS-v2/AMAS/files/ncbi/ncbigene2label_all_protein_coding.lzma


This file is too large, only the dictionary takes 99.4MB. I'll go with the organisms in Bigg models first.

In [100]:
bigg_organisms = pd.read_csv(bigg_model_dir + "bigg_organisms.csv")
taxids = bigg_organisms['tax_id'].unique().tolist()
taxids = [x for x in taxids if x and str(x).strip()] # Remove empty strings

# Only include rows where tax_id is in taxids
df['#tax_id'] = df['#tax_id'].astype(str)
bigg_df = df[df['#tax_id'].isin(taxids)]
bigg_df = bigg_df.map(str)
output_geneinfo_bigg = "ncbi_gene_info_bigg_organisms_protein_coding.csv"
bigg_df.to_csv(output_dir + output_geneinfo_bigg, index=False)

print(f"Number of organisms contained in NCBI gene info: {bigg_df['#tax_id'].nunique()}")
print(f"Number of rows: {len(bigg_df)}")
bigg_df.head()
print(f"All gene info of bigg organisms saved to {output_dir + output_geneinfo_bigg}")

Number of organisms contained in NCBI gene info: 18
Number of rows: 146842
All gene info of bigg organisms saved to /Users/luna/Desktop/CRBM/AMAS_proj/AMAS-v2/AMAS/files/ncbi/ncbi_gene_info_bigg_organisms_protein_coding.csv


In [101]:
output_gene2label = "ncbigene2label_bigg_organisms_protein_coding.lzma"
gene_id_to_symbol = dict(zip(bigg_df['GeneID'], bigg_df['Symbol']))
with open(os.path.join(output_dir,output_gene2label), 'wb') as handle:
    compress_pickle.dump(gene_id_to_symbol, handle, compression="lzma", set_default_extension=False)
print(f"GeneID to Symbol saved to {output_dir+output_gene2label}")

GeneID to Symbol saved to /Users/luna/Desktop/CRBM/AMAS_proj/AMAS-v2/AMAS/files/ncbi/ncbigene2label_bigg_organisms_protein_coding.lzma


The GeneID to Symbol dictionary for bigg organisms takes 502KB, which is fine.  
But the character count takes significant memory and casues kernel to crash.  
  
I'll save the character count for each organism separately.   
They will be located at the `files/charcount_gene_df_scaled` folder.

In [84]:
# Split the Synonyms column and include the Symbol in the list of synonyms
def expand_row(row):
    """
    Expand a row to include the Symbol and each of its synonyms as separate rows.
    """
    synonyms = row["Synonyms"].split('|') if pd.notna(row["Synonyms"]) else []
    synonyms.append(row["Symbol"])  # Add the Symbol as part of synonyms
    return [(row["#tax_id"], row["GeneID"], synonym) for synonym in synonyms]
# Function to calculate scaled character frequency
def scaled_char_frequency_row(row, char_columns):
    """
    Generate a scaled frequency row for each synonym.
    """
    synonym = str(row['Synonym']).lower()  # Convert to lowercase for case insensitivity
    char_counts = {char: synonym.count(char) for char in char_columns}  # Count each character
    total_count = sum(char_counts.values())  # Total characters
    vector_length = np.sqrt(sum(value ** 2 for value in char_counts.values()))  # Vector length (L2 norm)
    if vector_length == 0:
        # Avoid division by zero; all frequencies are 0 if the vector is empty
        scaled_counts = {char: 0 for char in char_columns}
    else:
        scaled_counts = {char: count / vector_length for char, count in char_counts.items()}  # Scale by vector length
    scaled_counts['tax_id'] = row['#tax_id']
    scaled_counts['synonym'] = row['Synonym']
    scaled_counts['ncbigene'] = row['GeneID']
    return scaled_counts
    
def process_taxid_file(tax_id, organism, input_df, char_columns, output_dir):
    """
    Process data for a specific tax_id, generate character frequency, and save to a compressed file.
    """
    # Filter rows for the specific tax_id
    taxid_df = input_df[input_df['#tax_id'] == tax_id]
    print(f"Number of genes for {tax_id} ({organism}): {len(taxid_df)}")
    # Expand the rows
    expanded_data = [item for _, row in taxid_df.iterrows() for item in expand_row(row)]
    expanded_df = pd.DataFrame(expanded_data, columns=["#tax_id", "GeneID", "Synonym"])
    
    # Compute character frequency
    char_freq_data = [scaled_char_frequency_row(row, char_columns) for _, row in expanded_df.iterrows()]
    char_freq_df = pd.DataFrame(char_freq_data)
    
    # Save to a compressed file
    output_file = os.path.join(output_dir, f"charcount_gene_df_scaled_{tax_id}.lzma")
    compress_pickle.dump(char_freq_df, output_file, compression="lzma")

def charcount_individual_organisms(df, bigg_organisms, output_dir):
    bigg_organisms_unique = bigg_organisms.drop_duplicates(subset='tax_id')

    # Define character columns (a-z and 0-9)
    letters = list(string.ascii_lowercase)
    numbers = list(string.digits)
    char_columns = letters + numbers

    # Initialize a list to store statistics
    stats = []

    # Iterate through each tax_id and process
    for _, organism_row in bigg_organisms_unique.iterrows():
        tax_id = str(organism_row['tax_id'])
        organism = organism_row['organism']
        
        # Filter rows for the specific tax_id
        taxid_df = df[df['#tax_id'] == tax_id]
        num_genes = len(taxid_df)
        stats.append({'tax_id': tax_id, 'organism': organism, 'num_genes': num_genes})

        if num_genes == 0:
            print(f"No genes found for tax_id {tax_id}({organism})")
            continue
        
        # Process the file for character frequency and save
        process_taxid_file(tax_id, organism, df, char_columns, output_dir)

    # Create a summary DataFrame
    stats_df = pd.DataFrame(stats)
    stats_file = os.path.join(output_dir, "organism_gene_statistics.csv")
    stats_df.to_csv(stats_file, index=False)
    print(f"Statistics saved to {stats_file}")
    return stats_df

bigg_organisms = pd.read_csv(bigg_model_dir + "bigg_organisms.csv")
output_dir = "/Users/luna/Desktop/CRBM/AMAS_proj/AMAS-v2/AMAS/files/ncbi/"
df = pd.read_csv(output_dir + "ncbi_gene_info_bigg_organisms_protein_coding.csv")
df['#tax_id'] = df['#tax_id'].astype(str)

charcount_individual_organisms(df, bigg_organisms, output_dir)
print(f"Character count for bigg organisms saved to {output_dir + 'charcount_gene_df_scaled_bigg_organisms.lzma'}")

Number of genes for 511145 (Escherichia coli str. K-12 substr. MG1655): 4288
Number of genes for 9606 (Homo sapiens): 20595
No genes found for tax_id 269797(Methanosarcina barkeri str. Fusaro)
No genes found for tax_id 269799(Geobacter metallireducens GS-15)
Number of genes for 5821 (Plasmodium berghei): 3
Number of genes for 1120755 (Plasmodium cynomolgi strain B): 5716
Number of genes for 36329 (Plasmodium falciparum 3D7): 5282
Number of genes for 5851 (Plasmodium knowlesi strain H): 5321
Number of genes for 126793 (Plasmodium vivax Sal-1): 3
No genes found for tax_id 405955(Escherichia coli APEC O1)
No genes found for tax_id 469008(Escherichia coli BL21(DE3))
No genes found for tax_id 595496(Escherichia coli BW2952)
No genes found for tax_id 199310(Escherichia coli CFT073)
Number of genes for 10029 (Cricetulus griseus): 22956
No genes found for tax_id 509173(Acinetobacter baumannii AYE)
No genes found for tax_id 272563(Clostridioides difficile 630)
No genes found for tax_id 574521(E

In [67]:
## REF_TAXID2LABEL
output_dir = "/Users/luna/Desktop/CRBM/AMAS_proj/AMAS-v2/AMAS/files/ncbi/"
stats_df = pd.read_csv(output_dir + "organism_gene_statistics.csv")
REF_TAXID2LABEL = dict(zip(stats_df['tax_id'], stats_df['organism']))

REF_DIR = "/Users/luna/Desktop/CRBM/AMAS_proj/AMAS-v2/AMAS/files/"
with open(REF_DIR + "taxid2label.lzma", 'wb') as f:
    compress_pickle.dump(REF_TAXID2LABEL, f, compression="lzma")
print(f"REF_TAXID2LABEL saved to {REF_DIR + 'taxid2label.lzma'}")

REF_TAXID2LABEL saved to /Users/luna/Desktop/CRBM/AMAS_proj/AMAS-v2/AMAS/files/taxid2label.lzma


### For individual organisms

In [72]:
# Split the Synonyms column and include the Symbol in the list of synonyms
def expand_row(row):
    """
    Expand a row to include the Symbol and each of its synonyms as separate rows.
    """
    synonyms = row["Synonyms"].split('|') if pd.notna(row["Synonyms"]) else []
    synonyms.append(row["Symbol"])  # Add the Symbol as part of synonyms
    return [(row["#tax_id"], row["GeneID"], synonym) for synonym in synonyms]

# Function to calculate scaled character frequency
def scaled_char_frequency_row(row, char_columns):
    """
    Generate a scaled frequency row for each synonym.
    """
    synonym = str(row['Synonym']).lower()  # Convert to lowercase for case insensitivity
    char_counts = {char: synonym.count(char) for char in char_columns}  # Count each character
    total_count = sum(char_counts.values())  # Total characters
    vector_length = np.sqrt(sum(value ** 2 for value in char_counts.values()))  # Vector length (L2 norm)
    if vector_length == 0:
        # Avoid division by zero; all frequencies are 0 if the vector is empty
        scaled_counts = {char: 0 for char in char_columns}
    else:
        scaled_counts = {char: count / vector_length for char, count in char_counts.items()}  # Scale by vector length
    scaled_counts['tax_id'] = row['#tax_id']
    scaled_counts['synonym'] = row['Synonym']
    scaled_counts['ncbigene'] = row['GeneID']
    return scaled_counts

# Process reference files
def process_reference_files(input_file, output_dir, output_gene2label, output_synonyms, output_charcount):
    ########### GeneID to Symbol
    columns_to_keep = ["#tax_id", "GeneID", "Symbol", "Synonyms"]
    df = pd.read_csv(input_file, compression='gzip', sep='\t', usecols=columns_to_keep)
    gene_id_to_symbol = dict(zip(df['GeneID'], df['Symbol']))
    compress_pickle.dump(gene_id_to_symbol, output_dir+output_gene2label, compression="lzma")
    print(f"GeneID to Symbol saved to {output_dir+output_gene2label}")
    
    # ########### GeneID to Symbol and Synonyms
    # gene_id_to_symbol_synonyms = {gene_id: (symbol, synonyms.split('|')) for gene_id, symbol, synonyms in zip(df['GeneID'], df['Symbol'], df['Synonyms'])}
    # compress_pickle.dump(gene_id_to_symbol_synonyms, output_dir+output_synonyms, compression="lzma")
    # print(f"GeneID to Symbol and Synonyms saved to {output_dir+output_synonyms}")
    
    ########### GeneID to Character Counts
    expanded_data = []
    for _, row in df.iterrows():
        expanded_data.extend(expand_row(row))
    expanded_df = pd.DataFrame(expanded_data, columns=["#tax_id", "GeneID", "Synonym"])

    # Define the characters for the first 36 columns
    letters = list(string.ascii_lowercase)  # a-z
    numbers = list(string.digits)  # 0-9
    char_columns = letters + numbers

    char_freq_data = [scaled_char_frequency_row(row, char_columns) for _, row in expanded_df.iterrows()]
    char_freq_df = pd.DataFrame(char_freq_data)
    compress_pickle.dump(char_freq_df, output_dir+output_charcount, compression="lzma")
    print(f"Scaled character frequency DataFrame saved to {output_dir+output_charcount}")
    return char_freq_df

In [46]:
# Ecoli-MG1655
input_file = "/Users/luna/Desktop/CRBM/AMAS_proj/AMAS-v2/AMAS/files/ncbi/Escherichia_coli_str._K-12_substr._MG1655.gene_info.gz"
output_dir = "/Users/luna/Desktop/CRBM/AMAS_proj/AMAS-v2/AMAS/files/ncbi/"
output_gene2label = "ncbigene2label_ecoli_mg1655.lzma"
output_charcount = 'charcount_gene_df_scaled_ecoli_mg1655.lzma'
process_reference_files(input_file, output_dir, output_gene2label, output_synonyms, output_charcount)

GeneID to Symbol saved to /Users/luna/Desktop/CRBM/AMAS_proj/AMAS-v2/AMAS/files/ncbi/ncbigene2label_ecoli_mg1655.lzma
Scaled character frequency DataFrame saved to /Users/luna/Desktop/CRBM/AMAS_proj/AMAS-v2/AMAS/files/ncbi/charcount_gene_df_scaled_ecoli_mg1655.lzma


,a,b,c,d,e,f,g,h,i,j,...,3,4,5,6,7,8,9,tax_id,synonym,ncbigene
0,0.0,0.0,0.333333,0.0,0.333333,0.0,0.0,0.0,0.0,0.0,...,0.0,0.333333,0.000000,0.000000,0.0,0.000000,0.666667,511145,ECK4299,944740
1,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.5,0.0,0.5,...,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,511145,yjhR,944740
2,0.0,0.0,0.333333,0.0,0.333333,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.333333,0.333333,0.0,0.000000,0.000000,511145,ECK0560,944741
3,0.5,0.0,0.000000,0.0,0.000000,0.5,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,511145,nfrA,944741
4,0.0,0.0,0.277350,0.0,0.277350,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,511145,ECK0001,944742
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13007,0.0,0.5,0.000000,0.0,0.000000,0.0,0.5,0.0,0.0,0.0,...,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,511145,ytgB,71004580
13008,0.0,0.0,0.377964,0.0,0.377964,0.0,0.0,0.0,0.0,0.0,...,0.0,0.377964,0.000000,0.377964,0.0,0.377964,0.000000,511145,ECK4680,71004581
13009,0.0,0.0,0.000000,0.0,0.500000,0.0,0.0,0.0,0.5,0.0,...,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,511145,ytiE,71004581
13010,0.0,0.0,0.377964,0.0,0.377964,0.0,0.0,0.0,0.0,0.0,...,0.0,0.377964,0.000000,0.377964,0.0,0.377964,0.000000,511145,ECK4681,71004582


In [47]:
# Human
input_file = "/Users/luna/Desktop/CRBM/AMAS_proj/AMAS-v2/AMAS/files/ncbi/Homo_sapiens.gene_info.gz"
output_dir = "/Users/luna/Desktop/CRBM/AMAS_proj/AMAS-v2/AMAS/files/ncbi/"
output_gene2label = "ncbigene2label_human.lzma"
output_charcount = 'charcount_gene_df_scaled_human.lzma'
process_reference_files(input_file, output_dir, output_gene2label, output_synonyms, output_charcount)

GeneID to Symbol saved to /Users/luna/Desktop/CRBM/AMAS_proj/AMAS-v2/AMAS/files/ncbi/ncbigene2label_human.lzma
Scaled character frequency DataFrame saved to /Users/luna/Desktop/CRBM/AMAS_proj/AMAS-v2/AMAS/files/ncbi/charcount_gene_df_scaled_human.lzma


,a,b,c,d,e,f,g,h,i,j,...,3,4,5,6,7,8,9,tax_id,synonym,ncbigene
0,0.577350,0.57735,0.0,0.0,0.0,0.0,0.00000,0.000000,0.0,0.0,...,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,9606,A1B,1
1,0.577350,0.57735,0.0,0.0,0.0,0.0,0.57735,0.000000,0.0,0.0,...,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,9606,ABG,1
2,0.577350,0.57735,0.0,0.0,0.0,0.0,0.57735,0.000000,0.0,0.0,...,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,9606,GAB,1
3,0.000000,0.00000,0.0,0.0,0.0,0.0,0.00000,0.316228,0.0,0.0,...,0.0,0.316228,0.0,0.000000,0.632456,0.0,0.0,9606,HYST2477,1
4,0.500000,0.50000,0.0,0.0,0.0,0.0,0.50000,0.000000,0.0,0.0,...,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,9606,A1BG,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
433435,0.500000,0.00000,0.0,0.0,0.0,0.0,0.00000,0.000000,0.0,0.0,...,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,741158,trnA,8923217
433436,0.000000,0.00000,0.0,0.0,0.0,0.0,0.00000,0.000000,0.0,0.0,...,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,741158,-,8923218
433437,0.000000,0.00000,0.5,0.0,0.0,0.0,0.00000,0.000000,0.0,0.0,...,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,741158,COX1,8923218
433438,0.000000,0.00000,0.0,0.0,0.0,0.0,0.00000,0.000000,0.0,0.0,...,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,741158,-,8923219


In [53]:
# Concatenate files for different taxonomies
output_dir = "/Users/luna/Desktop/CRBM/AMAS_proj/AMAS-v2/AMAS/files/ncbi/"
output_gene2label_combined = "ncbigene2label.lzma"
output_charcount_combined = "charcount_gene_df_scaled.lzma"

# gene2label files for E. coli and human
ecoli_gene2label = compress_pickle.load(output_dir + "ncbigene2label_ecoli_mg1655.lzma", compression="lzma")
human_gene2label = compress_pickle.load(output_dir + "ncbigene2label_human.lzma", compression="lzma")
combined_gene2label = {**ecoli_gene2label, **human_gene2label}
with open(output_dir + output_gene2label_combined, 'wb') as f:
    compress_pickle.dump(combined_gene2label, f, compression="lzma")
print(f"Combined gene2label dictionary saved to {output_dir + output_gene2label_combined}")

# charcount files for E. coli and human
ecoli_charcount = compress_pickle.load(output_dir + "charcount_gene_df_scaled_ecoli_mg1655.lzma", compression="lzma")
human_charcount = compress_pickle.load(output_dir + "charcount_gene_df_scaled_human.lzma", compression="lzma")
combined_charcount = pd.concat([ecoli_charcount, human_charcount])
with open(output_dir + output_charcount_combined, 'wb') as f:
    compress_pickle.dump(combined_charcount, f, compression="lzma")
print(f"Combined charcount DataFrame saved to {output_dir + output_charcount_combined}")

Combined gene2label dictionary saved to /Users/luna/Desktop/CRBM/AMAS_proj/AMAS-v2/AMAS/files/ncbi/ncbigene2label.lzma
Combined charcount DataFrame saved to /Users/luna/Desktop/CRBM/AMAS_proj/AMAS-v2/AMAS/files/ncbi/charcount_gene_df_scaled.lzma


## Test functions

In [2]:
one_fpath = "/Users/luna/Desktop/CRBM/AMAS_proj/Models/BiggModels/e_coli_core.xml"
recom = recommender.Recommender(libsbml_fpath=one_fpath)
genes = recom.getGeneIDs()
pred = recom.getGeneListRecommendation(pred_ids = genes,tax_id='511145', get_df=True)
pred


[        annotation  match score label
 G_b1241                              
 0           945695          1.0  purB
 1           945837          1.0  adhE
 2           948025          1.0  hdeA,
         annotation  match score label
 G_b0351                              
 0           945008          1.0  mhpF,
         annotation  match score label
 G_s0001                              
 0           944742        0.836  thrL
 1           944792        0.836  satP
 2           944824        0.836  mutT
 3           946586        0.836  rutD,
         annotation  match score label
 G_b1849                              
 0           946199          1.0  rstA
 1           946368          1.0  purT,
         annotation  match score label
 G_b3115                              
 0           947635          1.0  tdcD,
         annotation  match score label
 G_b2296                              
 0           946775          1.0  ackA,
         annotation  match score label
 G_b1276           

In [108]:
one_fpath = "/Users/luna/Desktop/CRBM/AMAS_proj/Models/BioModels/BIOMD0000000190.xml"
recom = recommender.Recommender(libsbml_fpath=one_fpath)
model = recom.sbml_document.getModel()
try:
    for i, gene in enumerate(model.getPlugin("fbc").getListOfGeneProducts(), start=1):
        if not gene.isSetMetaId():
            gene.setMetaId(generateMetaId(i))
except:
    pass
    

In [35]:
etype = 'genes'
tax_id = '511145'
model = recom.sbml_document.getModel()
TYPE_EXISTING_ATTR = {'species': recom.species.exist_annotation,
                      'reaction': recom.reactions.exist_annotation,
                      'genes': recom.genes.exist_annotation} 
ELEMENT_FUNC = {'species': model.getSpecies,
                'reaction': model.getReaction,
                'genes':model.getPlugin("fbc").getGeneProduct}
TYPE_LABEL = {'species': cn.REF_CHEBI2LABEL,
              'reaction': cn.REF_RHEA2LABEL,
              'genes':cn.REF_NCBI_GENE2LABEL}
for one_edf in pred[:5]:
  print("____________________")
  element_id = one_edf.index.name
  print(f"element_id: {element_id}")
  if one_edf.shape[0] == 0:
    continue
  annotations = list(one_edf['annotation'])
  print(f"annotations: {annotations}")
  match_scores = list(one_edf[cn.DF_MATCH_SCORE_COL])
  labels = list(one_edf['label'])
  print(f"labels: {labels}")
  # if there is existing annotation among predicted candidates;
  if element_id in TYPE_EXISTING_ATTR[etype].keys():
    existings = [1 if str(val) in TYPE_EXISTING_ATTR[etype][element_id] else 0 \
                  for idx, val in enumerate(one_edf['annotation'])]
    print(f"existings: {existings}")
    upd_annotation = ['keep' if val in TYPE_EXISTING_ATTR[etype][element_id] else 'ignore' \
                      for idx, val in enumerate(one_edf['annotation'])]
    print(f"upd_annotation: {upd_annotation}")
    print(f"list(one_edf['annotation']): {list(one_edf['annotation'])}")
    print(f"TYPE_EXISTING_ATTR[etype][element_id]: {[val for val in TYPE_EXISTING_ATTR[etype][element_id]]}")
    annotation2add_raw = [val for val in TYPE_EXISTING_ATTR[etype][element_id] \
                          if val not in list(one_edf['annotation'])]
    print(f"annotation2add_raw: {annotation2add_raw}")
    # only use existing annotation that exists in the label dictionaries
    annotation2add = [val for val in annotation2add_raw \
                      if val in TYPE_LABEL[etype].keys()]  
    print(f"annotation2add: {annotation2add}")
  # if there doesn't exist existing annotation among predicted candidates;
  else:
    existings = [0] * len(annotations)
    upd_annotation = ['ignore'] * len(annotations)
    annotation2add = []
  # handling existing annotations that were not predicted
  for new_anot in annotation2add:
    annotations.append(new_anot)
    match_scores.append(recom.getMatchScoreOfNCBIGENE(element_id, new_anot, tax_id))
    labels.append(cn.REF_NCBI_GENE2LABEL[new_anot])

____________________
element_id: G_b1241
annotations: ['945695', '945837', '948025']
labels: ['purB', 'adhE', 'hdeA']
existings: [0, 1, 0]
upd_annotation: ['ignore', 'keep', 'ignore']
list(one_edf['annotation']): ['945695', '945837', '948025']
TYPE_EXISTING_ATTR[etype][element_id]: ['945837']
annotation2add_raw: []
annotation2add: []
____________________
element_id: G_b0351
annotations: ['945008']
labels: ['mhpF']
existings: [1]
upd_annotation: ['keep']
list(one_edf['annotation']): ['945008']
TYPE_EXISTING_ATTR[etype][element_id]: ['945008']
annotation2add_raw: []
annotation2add: []
____________________
element_id: G_s0001
annotations: ['944742', '944792', '944824', '946586']
labels: ['thrL', 'satP', 'mutT', 'rutD']
____________________
element_id: G_b1849
annotations: ['946199', '946368']
labels: ['rstA', 'purT']
existings: [0, 1]
upd_annotation: ['ignore', 'keep']
list(one_edf['annotation']): ['946199', '946368']
TYPE_EXISTING_ATTR[etype][element_id]: ['946368']
annotation2add_raw: [